In [14]:
import pandas as pd

df = pd.read_csv('../dados/codigos_distritos_msp.csv', encoding='latin1', sep=';')
df.head(3)

,codigo_distrito,cod_ibge,distrito,regioes,codigo_subprefeitura,subprefeituras
0,80001,1,Água Rasa,Leste,25.0,Mooca
1,80002,2,Alto de Pinheiros,Oeste,11.0,Pinheiros
2,80003,3,Anhanguera,Norte,1.0,Perus


In [15]:
from geopy.geocoders import Nominatim
import pandas as pd

# Lista dos distritos de São Paulo (exemplo)
distritos = df['distrito']

# Inicialize o geolocalizador
geolocator = Nominatim(user_agent="geoapiExercises")

# Função para obter latitude e longitude
def obter_lat_long(distrito):
    try:
        location = geolocator.geocode(distrito + ", São Paulo, Brasil")
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Erro ao obter coordenadas para {distrito}: {e}")
        return None, None

# Crie uma lista para armazenar os resultados
resultados = []

# Obtenha coordenadas para cada distrito
for distrito in distritos:
    lat, long = obter_lat_long(distrito)
    resultados.append({
        "Distrito": distrito,
        "Latitude": lat,
        "Longitude": long
    })
    time.sleep(1)

# Crie um DataFrame para visualizar os resultados
df2 = pd.DataFrame(resultados)
df2 = df2.dropna()

In [17]:
df2.to_csv('../dados/distritos_lat_lon.csv')

In [19]:
from meteostat import Point, Daily
from datetime import datetime

# Definir datas diretamente no código
start = datetime(1950, 1, 1)
end = datetime(2024, 8, 31)

# Lista para armazenar os dados de cada distrito
all_data = []

# Iterar sobre as linhas do DataFrame e obter dados do Meteostat
for index, row in df2.iterrows():
    latitude = row['Latitude']
    longitude = row['Longitude']
    location = Point(latitude, longitude)
    
    # Obter dados diários do Meteostat
    data = Daily(location, start, end).fetch()
    
    # Adicionar uma coluna com o nome do distrito
    data['Distrito'] = row['Distrito']
    
    # Adicionar os dados à lista
    all_data.append(data)

# Concatenar todos os DataFrames em um único DataFrame
final_data = pd.concat(all_data)

# Resetar o índice do DataFrame final
final_data.reset_index(inplace=True)

final_data

,index,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun,Distrito
0,1950-01-01,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Água Rasa
1,1950-01-02,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Água Rasa
2,1950-01-03,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Água Rasa
3,1950-01-04,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,Água Rasa
4,1950-01-05,NaN,NaN,NaN,29.3,NaN,NaN,NaN,NaN,NaN,NaN,Água Rasa
...,...,...,...,...,...,...,...,...,...,...,...,...
2558622,2024-08-23,22.6,15.5,31.2,0.0,NaN,347.0,17.5,NaN,1013.5,NaN,Lajeado
2558623,2024-08-24,17.6,14.6,26.8,3.5,NaN,152.0,16.2,NaN,1017.9,NaN,Lajeado
2558624,2024-08-25,12.1,9.1,14.2,6.6,NaN,167.0,14.8,NaN,1023.8,NaN,Lajeado
2558625,2024-08-26,10.4,6.4,16.2,0.0,NaN,145.0,13.1,NaN,1025.0,NaN,Lajeado


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# Acessar as credenciais da AWS
access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")

In [ ]:
import boto3
from io import StringIO


s3 = boto3.client('s3',
                  aws_access_key_id=access_key_id,
                  aws_secret_access_key=secret_access_key,
                  region_name = 'sa-east-1')

csv_buffer = StringIO()
final_data.to_csv(csv_buffer, index=False)

# Inicializar o cliente do AWS S3
s3 = boto3.client('s3')

# Carregar o arquivo CSV para o S3
s3.put_object(Bucket='black-umbrella-fiap',
              Key='bronze/meteostat/dados_meteorologicos_meteostat.csv',
              Body=csv_buffer.getvalue())